In [1]:
import pyvisa

In [ ]:
rm = pyvisa.ResourceManager(); print(rm) # see directory for reource manager (?)
res = rm.list_resources(); print("Resources : ", res) # See which resources are available (USB etc.)
instrument = rm.open_resource(res[0]) # In my case the index 0 is the USB device
device = instrument.query("*IDN?") # see what the device name is
brand, model, sn, = instrument.query("*IDN?").split(",")
print(f"        Brand : {brand}",
      f"        Model : {model}",
      f"Serial/Number : {sn}"
      )

Resource Manager of Visa Library at C:\WINDOWS\system32\visa32.dll
Resources :  ('USB0::0x2A8D::0x1766::MY56311274::INSTR', 'ASRL3::INSTR', 'ASRL4::INSTR')


ValueError: too many values to unpack (expected 3)

'USB0::0x2A8D::0x1766::MY56311274::INSTR'

'KEYSIGHT TECHNOLOGIES,DSO-X 3024T,MY56311274,07.20.2017102614\n'